# CUDA入门

本文为入门CUDA的简单知识点的整理。

## 线程模型

GPU上的计算由大量的线程组成，因此有了三层的抽象概念——线程、线程块和线程块网络。

> thread -> block -> grid

计算最小的执行单元是一个线程，很多线程组成一个线程块，很多线程块组成一个线程块网格。

CUDA中有内置变量帮助我们识别线程在计算当中的位置，例如：

- `threadIdx.x`表示线程在一个线程块内的一维编号
- `blockIdx.x`表示当前的线程块在线程块网格中的编号
- `blockDim.x`表示在一个线程块中有多少个线程
- `gridDim.x`表示一个线程块网络中有多少个线程块
- `blockIdx.x * blockDim.x + threadIdx.x`计算出一个线程在全局中的唯一索引

## 内存模型

CUDA的内存模型和计算机的存储系统设计思想一样，为了速度采用分层的思想。

- 寄存器：最快，每个线程私有
- 共享内存：一个线程块中的所有线程共享，速度快，适合块内通信
- 全局内存：最慢，但是所有的线程都可以访问，容量大

## 核函数

核函数（Kernel Funtion）就是在GPU上运行的函数，通常用`__global__`声明，例如：

```c
__global__ void kernel(float* A, float* B, float* C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        C[idx] = A[idx] + B[idx];
    }
}
```

在调用时应遵循如下格式：

```c
kernel<<<blocks, threads_per_block>>>(A, B, C, N);
```

## Pytorch的C++/CUDA拓展

CUDA内核和PyTorch中的张量结合，需要用到PyTorch的C++ API。

## PyBind

PyBind11是一个C++11的库，用来把C++写的类/函数导出为Python模块。

例如，如果我们使用C++写了一个加法函数：

```cpp
int add (int x, int y) {
    return x + y
}
```

如果我们要在Python中使用，必须要借助PyBind，应该这样写：

```cpp
#include <pybind11/pybind11.h>
namespace py = pybind11;

int add(int a, int b) {
    return a + b;
}

// 暴露给 Python
PYBIND11_MODULE(example, m) {
    m.def("add", &add, "A function that adds two numbers");
}

```

## PyTorch的C++拓展

PyTorch的C++拓展就是基于PyBind11的，需要我们使用封装好的库：

```cpp
#include <torch/extension.h>
```

然后导入：

```cpp
PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("add", &add_cuda, "Elementwise Add (CUDA)");
}
```

需要深入理解C++与Python的语法特性，充分发挥两者的优势，在实际开发中实现高效的优化。


In [ ]:
%%writefile elementwise_fp32.cu
#include <torch/extension.h>
#include <cuda_runtime.h>

__global__ void elementwise_add_f32_kernel(const float* A, const float* B, float* C, int N) {
    int idx = blockIdx.x * blockDim.x + threadIdx.x;
    if (idx < N) {
        C[idx] = A[idx] + B[idx];
    }
}

void elementwise_add_f32(torch::Tensor A, torch::Tensor B, torch::Tensor C) {
    int N = A.numel();
    int threads = 256;
    int blocks = (N + threads - 1) / threads;
    elementwise_add_f32_kernel<<<blocks, threads>>>(
        A.data_ptr<float>(),
        B.data_ptr<float>(),
        C.data_ptr<float>(),
        N
    );
}

PYBIND11_MODULE(TORCH_EXTENSION_NAME, m) {
    m.def("elementwise_add_f32", &elementwise_add_f32, "Elementwise Add f32");
}


In [1]:
# 测试代码如下
import time
import torch
from torch.utils.cpp_extension import load

torch.set_grad_enabled(False)

# 编译并加载 CUDA 扩展
lib = load(
    name="elementwise_lib",
    sources=["elementwise_fp32.cu"],
    extra_cuda_cflags=["-O3"],
    extra_cflags=["-std=c++17"],
    verbose=True
)

def run_benchmark(func, a, b, out=None, iters=1000):
    # warmup
    for _ in range(10):
        func(a, b, out)
    torch.cuda.synchronize()

    start = time.time()
    for _ in range(iters):
        func(a, b, out)
    torch.cuda.synchronize()
    end = time.time()

    mean_time = (end - start) * 1000 / iters
    print(f"Time per call: {mean_time:.6f} ms")

# 测试数据
N = 1024 * 1024  # 一百万个元素
a = torch.randn(N, device="cuda", dtype=torch.float32)
b = torch.randn(N, device="cuda", dtype=torch.float32)
c = torch.zeros_like(a)

# 跑自定义 kernel
run_benchmark(lib.elementwise_add_f32, a, b, c)

# 跑 PyTorch 内置加法
run_benchmark(lambda x,y,z: torch.add(x,y,out=z), a, b, c)

# 对比结果
print("验证结果是否一致:", torch.allclose(a+b, c))


Using /home/shaneyale/.cache/torch_extensions/py312_cu124 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/shaneyale/.cache/torch_extensions/py312_cu124/elementwise_lib/build.ninja...
/home/shaneyale/miniconda3/envs/cu124/lib/python3.12/site-packages/torch/utils/cpp_extension.py:2059: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
Building extension module elementwise_lib...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)


[1/2] /usr/local/cuda-12.4/bin/nvcc --generate-dependencies-with-compile --dependency-output elementwise_fp32.cuda.o.d -DTORCH_EXTENSION_NAME=elementwise_lib -DTORCH_API_INCLUDE_EXTENSION_H -DPYBIND11_COMPILER_TYPE=\"_gcc\" -DPYBIND11_STDLIB=\"_libstdcpp\" -DPYBIND11_BUILD_ABI=\"_cxxabi1011\" -isystem /home/shaneyale/miniconda3/envs/cu124/lib/python3.12/site-packages/torch/include -isystem /home/shaneyale/miniconda3/envs/cu124/lib/python3.12/site-packages/torch/include/torch/csrc/api/include -isystem /home/shaneyale/miniconda3/envs/cu124/lib/python3.12/site-packages/torch/include/TH -isystem /home/shaneyale/miniconda3/envs/cu124/lib/python3.12/site-packages/torch/include/THC -isystem /usr/local/cuda-12.4/include -isystem /home/shaneyale/miniconda3/envs/cu124/include/python3.12 -D_GLIBCXX_USE_CXX11_ABI=0 -D__CUDA_NO_HALF_OPERATORS__ -D__CUDA_NO_HALF_CONVERSIONS__ -D__CUDA_NO_BFLOAT16_CONVERSIONS__ -D__CUDA_NO_HALF2_OPERATORS__ --expt-relaxed-constexpr -gencode=arch=compute_89,code=compu

Loading extension module elementwise_lib...


Time per call: 0.018693 ms
Time per call: 0.014354 ms
验证结果是否一致: True
